<a href="https://colab.research.google.com/github/Babitha23/Cyberbullying-detection/blob/main/Bert%20word%20embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [3]:
!pip install wordcloud
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from google.colab import drive
%matplotlib inline
import re
from collections import Counter
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
# Connect to google drive
drive.mount('/content/gdrive')

# Load data and set labels
data = pd.read_csv('gdrive/My Drive/Dissertation/final.csv', index_col='_unit_id')

Mounted at /content/gdrive


In [5]:
sample = data.iloc[0:5,:]

In [6]:
def cleancomments(text):
    tt = re.sub('\(created.at:\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d\)', '', str(text)) #replacing comment time
    tt1 = re.sub('<font color\=\"\#0066CC\"\>.*?\<\/font>', '', str(tt)) # removing html tags with username
    tt2 = re.sub('@[^\s]+', '', str(tt1)) #removing user tag
    #Remove unicode strings like "\u002c" and "x96"
    tmp = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', tt2) 
    temp = re.sub(r'[^\x00-\x7f]',r'',tmp)
    #removing url
    temp1 = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',temp)
    temp2 = re.sub(r'#([^\s]+)', r'\1', temp1) # removing hashtags
    #removing emoticons
    temp3 = re.sub(':\)|;\)|:-\)|<3|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', temp2)
    temp4 = ''.join([i for i in temp3 if not i.isdigit()]) #removing numbers
    temp5 = re.sub(r"( ?\.)\1+", '.', temp4) #replacing multiple . with single .
    temp6 = re.sub(r"( ?\!)+", '!', temp5) #replacing multiple ! with single !
    temp7 = re.sub(r"( ?\?)+", '?', temp6) #replacing multiple ? with single ?
    #temp8 = re.sub(r"(\_)+", '', temp7)
    temp8 = ' '.join( [w for w in temp7.split() if len(w)>1]) #removing single characters
    temp9 = re.sub(' +', ' ', temp8) #removing additional whitespaces
    return temp9

In [7]:
for i in range(len(sample)):
  sample['owner_cmnt'] = data['owner_cmnt'].apply(lambda x: cleancomments(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
sample.columns

Index(['question1', 'question2', 'clmn1', 'clmn10', 'clmn100', 'clmn101',
       'clmn102', 'clmn103', 'clmn104', 'clmn105',
       ...
       'clmn95', 'clmn96', 'clmn97', 'clmn98', 'clmn99', 'likes', 'owner_cmnt',
       'shared media', 'followed_by', 'follows'],
      dtype='object', length=202)

In [9]:
sample = sample.drop(['likes','shared media', 'followed_by', 'follows'], axis=1)

In [10]:
def replaceElongated(word):
    """ Replaces an elongated word with its basic form, unless the word exists in the lexicon """

    repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
    repl = r'\1\2\3'
    if wordnet.synsets(str(word)):
        return word
    repl_word = repeat_regexp.sub(repl, word)
    if repl_word != word:      
        return replaceElongated(repl_word)
    else:       
        return repl_word

In [11]:
import string
def removepunct(txt):
  remove = string.punctuation
  remove = remove.replace(".", "") # don't remove dot
  remove = remove.replace("!", "") # don't remove !
  remove = remove.replace("?", "") # don't remove ?
  pattern = r"[{}]".format(remove) # create the pattern
  text = re.sub(pattern, "", txt)
  return text

In [12]:
for col in sample.columns[2:]:
  for i in range(len(sample)):
    sample[col] = sample[col].apply(lambda x: removepunct(str(x)))

In [13]:
# for col in sample.columns[2:]:
#   for i in range(len(sample)):
#     sample[col] = sample[col].apply(lambda x: replaceElongated(str.split(x)))

In [14]:
sample.loc[702714442,'clmn94']

'Gemma sua lindaaaaa'

In [15]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [16]:
# for col in range(2,197):
#   for i in range(5):
#     if sample.iloc[i,col] is not np.NaN:
#       sent_l = sample.iloc[i,col].split()
#       for word in sent_l:
#         sample_df = sample.apply(lambda x: x.replace(regex=word, value=replaceElongated(word)))

In [17]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.2 MB/s 
     |████████████████████████████████| 7.6 MB 39.1 MB/s 
     |████████████████████████████████| 182 kB 49.5 MB/s 


In [18]:
import torch
from transformers import BertTokenizer, BertModel

import logging
logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
# Load pre-trained model tokenizer (vocabulary)
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True) # Whether the model returns all hidden-states.
# Put the model in "evaluation" mode,meaning feed-forward operation.
model.eval()

In [21]:
def tokenize(sentence, seg_id):
  # Add the special tokens.
  marked_text = "[CLS] " + sentence + " [SEP]"
  # Split the sentence into tokens.
  tokenized_text = tokenizer.tokenize(marked_text)
  # Map the token strings to their vocabulary indices.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  # Mark each of the tokens as belonging to sentence "seg_id".
  segments_ids = [seg_id] * len(tokenized_text)
  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  return {'tokens_tensor':tokens_tensor, 'segments_tensors': segments_tensors}

In [22]:
#copying only the structure of sample dataframe
sample_c = pd.DataFrame().reindex_like(sample)

In [23]:
s = sample.replace('nan', 'a')
sentences = s.drop(['question1','question2'], axis=1).to_numpy()
sentences[0][1]

'larry'

In [24]:
def embed(sentence):
  a=0
  tempdict = tokenize(sentence, a)
  with torch.no_grad():
    outputs = model(tempdict['tokens_tensor'], tempdict['segments_tensors'])
    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
  a+=1
  return token_embeddings

In [25]:
sample.columns[3:]

Index(['clmn10', 'clmn100', 'clmn101', 'clmn102', 'clmn103', 'clmn104',
       'clmn105', 'clmn106', 'clmn107', 'clmn108',
       ...
       'clmn91', 'clmn92', 'clmn93', 'clmn94', 'clmn95', 'clmn96', 'clmn97',
       'clmn98', 'clmn99', 'owner_cmnt'],
      dtype='object', length=195)

In [26]:
# embedding = {}
# for i in range(len(sentences)):
#   for j in range(len(sentences[i])):
#     embedding[sentences[i][j]] = embed(sentences[i][j])

for col in sample.columns[2:]:
  sample[col] = sample[col].apply(lambda x: embed(x))

In [27]:
sample.iloc[0,2].size() #[number of tokens, 13 layers, 768 features]

torch.Size([6, 13, 768])

In [28]:
sample.sample(2)

,question1,question2,clmn1,clmn10,clmn100,clmn101,clmn102,clmn103,clmn104,clmn105,...,clmn91,clmn92,clmn93,clmn94,clmn95,clmn96,clmn97,clmn98,clmn99,owner_cmnt
_unit_id,,,,,,,,,,,,,,,,,,,,,
702714442,noneAgg,noneBll,"[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....",...,"[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0...."
702714443,aggression,bullying,"[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....",...,"[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0....","[[[tensor(0.1686), tensor(-0.2858), tensor(-0...."


In [29]:
# Stores the token vectors, with shape [<num of tokens> x 768]
#token_vecs_sum = []
# each `embedding.values` is a [<num of tokens> x 12 x 768] tensor.
# For each token in the sentence...

def vec(embedd):
  token_vecs_sum = []
  for token in embedd:
    # `token` is a [12 x 768] tensor
    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)
  return token_vecs_sum

for col in sample.columns[2:]:
  sample[col] = sample[col].apply(lambda x: vec(x))

In [30]:
print("Number of tokens:", len(sample.iloc[0,2]))
print("Number of features:", sample.iloc[0,2][0].size())

Number of tokens: 6
Number of features: torch.Size([768])


In [31]:
sample.sample(2)

,question1,question2,clmn1,clmn10,clmn100,clmn101,clmn102,clmn103,clmn104,clmn105,...,clmn91,clmn92,clmn93,clmn94,clmn95,clmn96,clmn97,clmn98,clmn99,owner_cmnt
_unit_id,,,,,,,,,,,,,,,,,,,,,
702714440,noneAgg,noneBll,"[[tensor(0.1172), tensor(1.3275), tensor(2.317...","[[tensor(-1.8747), tensor(-0.4204), tensor(0.1...","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....",...,"[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(1.6468), tensor(3.6387), tensor(-0.11..."
702714442,noneAgg,noneBll,"[[tensor(1.3486), tensor(1.8491), tensor(-0.89...","[[tensor(-2.6285), tensor(1.1504), tensor(2.48...","[[tensor(1.1686), tensor(2.4881), tensor(1.809...","[[tensor(-2.7789), tensor(-0.0074), tensor(-0....","[[tensor(1.8391), tensor(1.1013), tensor(1.194...","[[tensor(1.2043), tensor(0.7256), tensor(2.042...","[[tensor(2.1311), tensor(0.1135), tensor(2.434...","[[tensor(0.4349), tensor(2.3690), tensor(0.427...",...,"[[tensor(0.1797), tensor(3.1708), tensor(1.065...","[[tensor(2.3005), tensor(3.4559), tensor(0.643...","[[tensor(-0.5102), tensor(3.7601), tensor(2.00...","[[tensor(0.3700), tensor(1.6582), tensor(0.809...","[[tensor(0.6715), tensor(0.7315), tensor(0.534...","[[tensor(1.0799), tensor(1.3855), tensor(1.149...","[[tensor(-0.8237), tensor(2.4991), tensor(0.80...","[[tensor(-0.7659), tensor(-1.9407), tensor(0.2...","[[tensor(0.0387), tensor(-0.5235), tensor(1.44...","[[tensor(0.9526), tensor(2.1736), tensor(1.042..."


In [35]:
print("Number of tokens:", len(sample.iloc[0,2]))
print("Number of features:", sample.iloc[0,2][0].size())
print("Shape: {}*{}".format(len(sample.iloc[0,2]),len(sample.iloc[0,2][0])))

Number of tokens: 6
Number of features: torch.Size([768])
Shape: 6*768


In [ ]:
count = 0
for i in range(5):
  count = count + len(sentences[i])

print("total number of sentences including NA:", count) #196 columns * 5 rows = 980

total number of sentences including NA: 980


In [ ]:
# count1 = 0
# for i in range(5):
#   #count1 += np.count_nonzero(sentences[i] == 'a')
#   count1 = count1 + (sentences[i] != 'a').sum()
# print("Senternces without NA:", count1)

In [ ]:
# conditions = [
#     df['gender'].eq('male') & df['pet1'].eq(df['pet2']),
#     df['gender'].eq('female') & df['pet1'].isin(['cat', 'dog'])
# ]

# choices = [5,5]

# df['points'] = np.select(conditions, choices, default=0)

In [ ]:
# s = sample.replace(np.NaN, '')
# n = s.drop(['question1','question2'], axis=1).to_numpy()
# n

In [ ]:
## create label and sentence list
# sentences = n

In [ ]:
# ## Import BERT tokenizer, that is used to convert our text into tokens that corresponds to BERT library
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
# input_ids = [tokenizer.encode(sent, add_special_tokens=True,padding==True) for sent in sentences]
# labels = s.question1.values

# print("Actual sentence before tokenization: ",sentences[2])
# print("Encoded Input from dataset: ",input_ids[2])

# ## Create attention mask
# attention_masks = []
# ## Create a mask of 1 for all input tokens and 0 for all padding tokens
# attention_masks = [[float(i>0) for i in seq] for seq in input_ids]
# print(attention_masks[2])

In [ ]:
# newsent = np.array([])
# for i in range(5):
#   newsent[i] = np.delete(sentences[i], np.where(sentences[i] == 'a'))

In [ ]:
# token_vec = {}
# for text, embedd in embedding.items():
#   token_vecs_sum = []
#   for token in embedd:
#     # `token` is a [12 x 768] tensor
#     # Sum the vectors from the last four layers.
#     sum_vec = torch.sum(token[-4:], dim=0)
#     # Use `sum_vec` to represent `token`.
#     token_vecs_sum.append(sum_vec)
#     token_vec[text] = token_vecs_sum

#print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))